In [6]:
import pandas as pd
import numpy as np
df = pd.read_excel(r'C:\Users\PULAPA YESHWANTH\Downloads\pumpkin_seed_dataset\Pumpkin_Seeds_Dataset\Pumpkin_Seeds_Dataset.xlsx')


Paper Link : https://link.springer.com/article/10.1007/s10722-021-01226-0  
Kaggle Link : https://www.kaggle.com/datasets/muratkokludataset/pumpkin-seeds-dataset


In [7]:
df.head()

,Area,Perimeter,Major_Axis_Length,Minor_Axis_Length,Convex_Area,Equiv_Diameter,Eccentricity,Solidity,Extent,Roundness,Aspect_Ration,Compactness,Class
0,56276,888.242,326.1485,220.2388,56831,267.6805,0.7376,0.9902,0.7453,0.8963,1.4809,0.8207,Çerçevelik
1,76631,1068.146,417.1932,234.2289,77280,312.3614,0.8275,0.9916,0.7151,0.8440,1.7811,0.7487,Çerçevelik
2,71623,1082.987,435.8328,211.0457,72663,301.9822,0.8749,0.9857,0.7400,0.7674,2.0651,0.6929,Çerçevelik
3,66458,992.051,381.5638,222.5322,67118,290.8899,0.8123,0.9902,0.7396,0.8486,1.7146,0.7624,Çerçevelik
4,66107,998.146,383.8883,220.4545,67117,290.1207,0.8187,0.9850,0.6752,0.8338,1.7413,0.7557,Çerçevelik


solidity is synonym for compactness eccentricity is synonym for aspect ratio The measure of “roundness” of an ellipse is given by its eccentricityewhere the value ofeis always between 0 and 1. For a circlee= 0. The higher the value ofe, the higher is thedeviation of the ellipse from its roundness.

In [8]:
new_df = df.drop(['Aspect_Ration','Compactness'],axis='columns')

In [9]:
pd.isnull(new_df).sum()

Area                 0
Perimeter            0
Major_Axis_Length    0
Minor_Axis_Length    0
Convex_Area          0
Equiv_Diameter       0
Eccentricity         0
Solidity             0
Extent               0
Roundness            0
Class                0
dtype: int64

In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
new_df['Class'] = le.fit_transform(new_df['Class'])
new_df['Class']

0       0
1       0
2       0
3       0
4       0
       ..
2495    1
2496    1
2497    1
2498    1
2499    1
Name: Class, Length: 2500, dtype: int32

In [11]:
x = new_df.drop('Class',axis='columns').values
y = new_df.Class.values
#while using stratified kfold we get error because It have to be Numpy vectorized not a data frame when you load in your data.
#so over come it we use values
from sklearn.preprocessing import MinMaxScaler,StandardScaler
#scaler = MinMaxScaler()
scaler = StandardScaler()
x = scaler.fit_transform(x)


# CROSS_VAL_SCORE

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score

In [22]:
logistic = list(cross_val_score(LogisticRegression(),x,y,cv=10))
print("Cross Validation for Logistic Regression"+str(np.average(logistic)))

svm = list(cross_val_score(SVC(C=1000),x,y,cv=10))
print("Cross Validation for SVM"+str(np.average(svm)))

rf = list(cross_val_score(RandomForestClassifier(criterion='gini'),x,y,cv=10))
print("Cross Validation for Random Forest"+str(np.average(rf)))

knn = list(cross_val_score(KNeighborsClassifier(n_neighbors=10),x,y,cv=10))
print("Cross Validation for KNearest"+str(np.average(knn)))

mlp = list(cross_val_score(MLPClassifier(random_state=1,max_iter=100),x,y,cv=10))
print("Cross Validation for MLP"+str(np.average(mlp)))

Cross Validation for Logistic Regression0.8779999999999999
Cross Validation for SVM0.8724000000000001
Cross Validation for Random Forest0.8832000000000001
Cross Validation for KNearest0.8756


C:\Users\PULAPA YESHWANTH\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\PULAPA YESHWANTH\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value

# Tunning Parameters

In [27]:
ln = [x for x in range(2,20)]
model_params = {
    'svm': {
        'model': SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20,1000,1500,2000,2500],
            #'kernel': ['rbf']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,6,7,8,9,10],
             'criterion':['gini']
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    },
    'knn':{
        'model':KNeighborsClassifier(),
        'params':{
            'n_neighbors':ln
        }
    },
    'mlp':{
        'model': MLPClassifier(),
        'params': {
            'random_state': [1], 'max_iter':[100]
            
        }
    }
}


In [28]:
from sklearn.model_selection import GridSearchCV

scores = []

for model_name, mp in model_params.items():
    print(model_name)
    print(mp)
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(x, y)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })

print(scores)
    

svm
{'model': SVC(gamma='auto'), 'params': {'C': [1, 10, 20, 1000, 1500, 2000, 2500]}}
random_forest
{'model': RandomForestClassifier(), 'params': {'n_estimators': [1, 5, 6, 7, 8, 9, 10], 'criterion': ['gini']}}
logistic_regression
{'model': LogisticRegression(solver='liblinear'), 'params': {'C': [1, 5, 10]}}
knn
{'model': KNeighborsClassifier(), 'params': {'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]}}


C:\Users\PULAPA YESHWANTH\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\PULAPA YESHWANTH\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value

C:\Users\PULAPA YESHWANTH\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\PULAPA YESHWANTH\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value

C:\Users\PULAPA YESHWANTH\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\PULAPA YESHWANTH\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value

C:\Users\PULAPA YESHWANTH\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\PULAPA YESHWANTH\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value

...

C:\Users\PULAPA YESHWANTH\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\PULAPA YESHWANTH\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value

mlp
{'model': MLPClassifier(), 'params': {'random_state': [1], 'max_iter': [100]}}


C:\Users\PULAPA YESHWANTH\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\PULAPA YESHWANTH\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\PULAPA YESHWANTH\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\PULAPA YESHWANTH\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged ye

[{'model': 'svm', 'best_score': 0.8876000000000002, 'best_params': {'C': 20}}, {'model': 'random_forest', 'best_score': 0.8844000000000001, 'best_params': {'criterion': 'gini', 'n_estimators': 9}}, {'model': 'logistic_regression', 'best_score': 0.8788, 'best_params': {'C': 10}}, {'model': 'knn', 'best_score': 0.8804000000000001, 'best_params': {'n_neighbors': 18}}, {'model': 'mlp', 'best_score': 0.8836, 'best_params': {'max_iter': 100, 'random_state': 1}}]


C:\Users\PULAPA YESHWANTH\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


In [29]:
pd.DataFrame(scores,columns=['model','best_score','best_params'])

,model,best_score,best_params
0,svm,0.8876,{'C': 20}
1,random_forest,0.8844,"{'criterion': 'gini', 'n_estimators': 9}"
2,logistic_regression,0.8788,{'C': 10}
3,knn,0.8804,{'n_neighbors': 18}
4,mlp,0.8836,"{'max_iter': 100, 'random_state': 1}"


svm kernel=linear is computationally expensive. taking very long time to compute without proper scaling

In [ ]:
normal implementation of mlp without cross validation.

In [19]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)
mlp = MLPClassifier(random_state=1,max_iter=100)
mlp.fit(x_train,y_train)
y_predict = mlp.predict(x_test)



C:\Users\PULAPA YESHWANTH\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


<bound method ClassifierMixin.score of MLPClassifier(max_iter=100, random_state=1)>

In [23]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_predict)

0.862